# Analyze Vdbench Logs

In [1]:
# S3_LOGS_URLS = ['412URL', '413URL'] - for comparing 2 artifacts 
S3_LOGS_URLS = ['']
# Compare versions
versions = ['412', '413']

# Prerequsite

In [2]:

# SET log level
# logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)
logging.info("Prerequisite")

# extract credentials
import configparser
config = configparser.ConfigParser()
config.read('logs.conf')
username = config.get("data", 'user') 
password = config.get("data", 'password')


# Update fedora
!dnf update -y


from benchmark_runner.jupyterlab.templates.logs_operations.logs_operations import LogsOperations
from benchmark_runner.jupyterlab.templates.analyze_workloads.analyze_vdbench_logs.analyze_vdbench_logs import AnalyzeVdbenchLogs

# Logs operations
logs_operations_obj = []
analyze_vdbench_logs_obj = []
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    logs_operations = LogsOperations(s3_logs_url=S3_LOGS_URL)
    logs_operations_obj.append(logs_operations)

# Logs cleanup
logs_operations_obj[0].cleanup()

for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_vdbench_logs = AnalyzeVdbenchLogs(s3_logs_url=S3_LOGS_URL)
    analyze_vdbench_logs_obj.append(analyze_vdbench_logs)

# Output clean output display
analyze_vdbench_logs_obj[0].clear_output()

Last metadata expiration check: 0:04:15 ago on Mon Jul  3 12:53:15 2023.
Dependencies resolved.
Nothing to do.
Complete!


## Download S3 logs

In [ ]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    logs_operations_obj[i].download_s3_logs(username=username, password=password)

IntProgress(value=0, description='Downloading', max=235568633)

  0%|          | 0/235568633 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

## Untar logs & chmod 

In [4]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    logs_operations_obj[i].untar_and_chmod_logs()

# Get vdbench log files

In [5]:
csv_log_files=[]
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    csv_log_files.append(analyze_vdbench_logs_obj[i].get_vdbench_csv_log_files())

# Display Dataframe

In [6]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_vdbench_logs_obj[i].display_df()

display file: vdbench-vm-28345f0a_result.csv


,Run,Xfersize,Threads,Reqrate,Rate,Resp,Resp_std,Resp_max,MB/sec,MB_read,...,Access_resp,Access_resp_std,Access_resp_max,Compratio,Dedupratio,cpu_used,cpu_user,cpu_kernel,cpu_wait,cpu_idle
1,oltp1,4095,8,9999988,22534.4000,0.7995,0.0,0.0,88.0250,43.8943,...,0,0,0,2,NaN,22.8333,6.3225,16.5108,0,0
2,oltp2,19496,8,9999988,23356.2972,0.9813,0.0,0.0,434.2699,263.1401,...,0,0,0,2,NaN,26.7128,7.6864,19.0264,0,0
3,oltphw,19487,8,9999988,18787.6639,1.0986,0.0,0.0,349.1591,123.2808,...,0,0,0,2,NaN,23.5801,7.4615,16.1186,0,0
4,odss2,53585,8,9999988,11453.0972,0.9924,0.0,0.0,585.2917,514.1232,...,0,0,0,2,NaN,14.7031,4.2828,10.4203,0,0
5,odss128,41500,16,9999988,21469.5556,1.3976,0.0,0.0,849.7215,690.1582,...,0,0,0,2,NaN,28.8035,8.7117,20.0918,0,0
6,4kb_cache_read_32threads,4096,32,9999988,59390.5194,0.5375,0.0,0.0,231.9942,231.9942,...,0,0,0,2,NaN,31.6163,5.5989,26.0175,0,0
7,64kb_cache_read_4threads,65536,4,9999988,6004.3917,0.6620,0.0,0.0,375.2748,375.2748,...,0,0,0,2,NaN,18.0055,1.9885,16.0170,0,0
8,4kb_read_16threads,4095,16,9999988,26512.5444,0.6009,0.0,0.0,103.5646,103.5646,...,0,0,0,2,NaN,19.6946,4.9104,14.7842,0,0
9,64kb_read_4threads,65536,4,9999988,4776.4806,0.8310,0.0,0.0,298.5300,298.5300,...,0,0,0,2,NaN,8.9846,2.1485,6.8360,0,0
10,4kb_cache_write_32threads,4095,32,9999988,49768.3222,0.6417,0.0,0.0,194.4075,0.0000,...,0,0,0,2,NaN,8.8202,1.6956,7.1247,0,0


display file: vdbench-vm-6490012b_result.csv


,Run,Xfersize,Threads,Reqrate,Rate,Resp,Resp_std,Resp_max,MB/sec,MB_read,...,Access_resp,Access_resp_std,Access_resp_max,Compratio,Dedupratio,cpu_used,cpu_user,cpu_kernel,cpu_wait,cpu_idle
1,oltp1,4095,8,9999988,23239.8444,0.7749,0.0,0.0,90.7806,45.3073,...,0,0,0,2,NaN,23.6191,6.7847,16.8344,0,0
2,oltp2,19488,8,9999988,23950.0194,0.9643,0.0,0.0,445.1298,269.8738,...,0,0,0,2,NaN,27.8577,8.1767,19.6810,0,0
3,oltphw,19472,8,9999988,19674.4361,1.0393,0.0,0.0,365.3637,128.9590,...,0,0,0,2,NaN,24.5580,7.9609,16.5971,0,0
4,odss2,53604,8,9999988,11264.9889,1.0018,0.0,0.0,575.8798,506.1059,...,0,0,0,2,NaN,14.7899,4.6376,10.1522,0,0
5,odss128,41521,16,9999988,21222.2917,1.3976,0.0,0.0,840.3535,682.7212,...,0,0,0,2,NaN,27.4848,8.3746,19.1102,0,0
6,4kb_cache_read_32threads,4095,32,9999988,59458.8639,0.5368,0.0,0.0,232.2610,232.2610,...,0,0,0,2,NaN,34.7452,6.1224,28.6228,0,0
7,64kb_cache_read_4threads,65535,4,9999988,5740.4667,0.6929,0.0,0.0,358.7788,358.7788,...,0,0,0,2,NaN,19.3144,1.9883,17.3261,0,0
8,4kb_read_16threads,4095,16,9999988,27225.0056,0.5851,0.0,0.0,106.3476,106.3476,...,0,0,0,2,NaN,19.9651,5.1873,14.7778,0,0
9,64kb_read_4threads,65535,4,9999988,4599.2500,0.8638,0.0,0.0,287.4524,287.4524,...,0,0,0,2,NaN,8.8565,2.1529,6.7036,0,0
10,4kb_cache_write_32threads,4096,32,9999988,53086.8056,0.6016,0.0,0.0,207.3703,0.0000,...,0,0,0,2,NaN,9.7774,1.8203,7.9571,0,0


## Vdbench latency (sec)

In [7]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_vdbench_logs_obj[i].display_latency_horizontal()

Loading BokehJS ...

Loading BokehJS ...

## Vdbench IOPS

In [8]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_vdbench_logs_obj[i].display_iops_horizontal()

Loading BokehJS ...

Loading BokehJS ...

# Compare Vdbench latency (sec)

In [9]:
if len(S3_LOGS_URLS)>1:
    analyze_vdbench_logs_obj[0].compare_latency(csv_log_files1=csv_log_files[0], csv_log_files2=csv_log_files[1], legend_label1=versions[0], legend_label2=versions[1])

Loading BokehJS ...

Loading BokehJS ...

# Compare Vdbench IOPS

In [10]:
if len(S3_LOGS_URLS)>1:
    analyze_vdbench_logs_obj[0].compare_iops(csv_log_files1=csv_log_files[0], csv_log_files2=csv_log_files[1], legend_label1=versions[0], legend_label2=versions[1])

Loading BokehJS ...

Loading BokehJS ...